<a href="https://colab.research.google.com/github/NaderJS/hacknotebooks/blob/master/UmojaHack2021BioChallenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DeepChain Antibody Classification Challenge
### Evaluating neutralising antibodies for the next influenza pandemic using the DeepChain™ platform
Authors : Mohamed Jedidi (meds.jedidi@instadeep.com), Marcin Skwark (m.skwark@instadeep.com), Nicolas Lopez Carranza  (n.lopezcarranza@instadeep.com). Any Issues running this notebook feeel free to email us. 

Welcome to the challenge. In this notebook we will create a neural network that is able to estimate the binding energy between an antibody and Influenza HA protein



<img src="influenza-infection2.png" width="600">

## How we will build a scorer for Antibody Binding?
The antibody sequence is 221 amino acids long and will be difficult to learn from the entire sequence. Hence we will explore in DeepChain which are the regions on the antibody that are in contact with the receptor binding domain of the virus. These regions are called [CDR regions](https://en.wikipedia.org/wiki/Complementarity-determining_region) and DeepChains shows them in green

<img src="deepchain.png" width="800">


We already extracted these 3 regions for you but feel free to use the entire sequence as input for your model. 

Enough said. Lets get our hands dirty and build a simple biLSTM model!

## Binding scorer example

Download relevant Data

In [ ]:
from google.colab  import drive

drive.mount("/content/gdrive")



Mounted at /content/gdrive


In [ ]:
!cp "/content/gdrive/MyDrive/UHDeepChain.zip" .

In [ ]:
!unzip  -P lpdsv "UHDeepChain.zip"

Archive:  UHDeepChain.zip
   creating: DeepChain/
  inflating: DeepChain/SampleSubmission.csv  
  inflating: DeepChain/CDR2_ProtBert_embeddings.csv  
  inflating: DeepChain/CDR3_ProtBert_embeddings.csv  
  inflating: DeepChain/train.csv     
  inflating: DeepChain/UmojaHack2021BioChallenge.ipynb  
  inflating: DeepChain/influenza-infection2.png  
  inflating: DeepChain/deepchain.png  
   creating: DeepChain/.ipynb_checkpoints/
  inflating: DeepChain/.ipynb_checkpoints/DataCheck-checkpoint.ipynb  
  inflating: DeepChain/CDR1_ProtBert_embeddings.csv  
  inflating: DeepChain/test.csv      


In [ ]:
import pandas as  pd 
import numpy as np 
import tensorflow as tf
import os 
import urllib
import datetime

from tqdm  import tqdm
import random
import torch
from torch import optim
from torch import nn
from torch.nn import functional as F
if torch.cuda.is_available():  
  dev = "cuda:0" 
  print("gpu up")
else:  
  dev = "cpu"  
device = torch.device(dev)

from sklearn.model_selection import train_test_split
os.environ["CUDA_VISIBLE_DEVICES"] = "3" # change it to "0" if yo have only one gpu or the gpu numbe  that you would like to use 
tf.get_logger().setLevel('ERROR')

gpu up


In [ ]:
torch.manual_seed(42)
torch.cuda.manual_seed(42)
random.seed(42)

In [ ]:
data = pd.read_csv("DeepChain/train.csv")

In [ ]:
# display the first 5 rows from the train file 
data.head()

,ID,sequence,CDR1,CDR2,CDR3,binding
0,train_seq_0,QVQLKEHGPGLVNPSQSLSVTCSVSGFLLISNGVHWVRQPPGKGLE...,LLISN,WAGGNTN,YDYDNFTY,-11.36922
1,train_seq_1,QVQLKEYGPGLVAPSQSLSITCTVSGFLLISNGVHWVRQPPGKGLE...,LLISN,WAGGMTA,YCYDVFYY,-14.23689
2,train_seq_2,QVQLKESGGGLVAPSQSLSITCTVSGFLLTSAGVHWVRQPPGKGLH...,LLTSA,WAGGYVN,YDYDHFYY,-10.90296
3,train_seq_3,QVQLKESGPGLVAPSSSLSDTCEVSGFLLQSHGVHWVRQPPGKGLE...,LLQSH,WAGGNTN,HDADCFYY,-9.19866
4,train_seq_4,QCQLKESGPGMVAPSQSLSITCTVSGFLLTSNGVHWVRQPPGKGLE...,LLTSN,WAGGHTN,YDYDRFYY,-11.25864


Split the train file to 90% train and 10% validation 

In [ ]:
len(voc_set)

23

In [ ]:
# create dictionary to map Amino acids to integer
voc_set=set(['P', 'V', 'I', 'K', 'N', 'B', 'F', 'Y', 'E', 'W', 'R', 'D', 'X', 'S', 'C', 'U', 'Q', 'A', 'M', 'H', 'L', 'G', 'T'])
voc_set_map={ k:v for k , v in zip(voc_set,range(1,len(voc_set)+1))}

In [ ]:
train 

,ID,sequence,CDR1,CDR2,CDR3,binding
3996,[3996],QSQLRESGPGLVAPSQSLSITCTVSGFLLVSNGVHWVSQPPGGGCE...,"[19, 19, 23, 22, 6]","[10, 8, 15, 15, 21, 11, 6, 13, 14, 13, 14, 23,...","[13, 14, 13, 14, 23, 15, 13, 13]",-12.72924
25846,[25846],QVQLKEEGPGLVAPSQSLSITCTVSGFLLISNGVHWVRQPPGKGLE...,"[19, 19, 16, 22, 6]","[10, 8, 15, 15, 14, 11, 6]","[13, 14, 13, 14, 3, 15, 13, 13]",-8.93703
31731,[31731],QVLLKESGPGLVAPSQSLSITCTVSGFLLQSNGVHWVRQGPGYGLE...,"[19, 19, 20, 22, 6]","[10, 8, 15, 15, 6, 11, 6]","[13, 14, 13, 14, 9, 9, 13, 13]",-11.29854
448,[448],QCALKESGPGLVAPSQSLSITCTVSGFLLSSNGVHWVRQPPGKGLE...,"[19, 19, 22, 22, 6]","[10, 8, 15, 15, 6, 11, 8]","[13, 3, 13, 14, 23, 13, 13, 13]",-11.81610
25903,[25903],QVQLKESGPGLVAPNQSLSITCLVSGFLLIKNGVHWVAQPPGKGLS...,"[19, 19, 16, 18, 6]","[10, 8, 15, 15, 6, 11, 22]","[13, 14, 13, 14, 23, 9, 13, 13]",-12.67680
...,...,...,...,...,...,...
29375,[29375],TVQLHECGPGLVAPSQSLSITCTVSGFLLASNGVHWVRQPPGKGLE...,"[19, 19, 8, 22, 6]","[10, 8, 15, 15, 6, 3, 6]","[13, 14, 13, 14, 23, 8, 13, 13]",-9.71109
17950,[17950],QVQLKESGIGLVAPSKSLSITCTVSGFLLISNGVHWVRQPPGKGLI...,"[19, 19, 16, 22, 6]","[10, 8, 15, 15, 2, 11, 6]","[13, 14, 13, 14, 22, 3, 13, 13]",-11.02665
26074,[26074],QVQLIESGPGCVAPGQSLSITCTVSGFLLISAGVHWVRQPPGKGLE...,"[19, 19, 16, 22, 8]","[10, 8, 15, 15, 2, 11, 6]","[19, 14, 14, 14, 23, 9, 13, 13]",-11.08821
37200,[37200],QVQLHELGPGLVAPSQSLSITCTVSGFLLIGNGVHWVRQPPGKGLE...,"[19, 19, 16, 15, 6]","[10, 8, 15, 15, 6, 11, 22]","[13, 14, 13, 14, 17, 9, 13, 13]",-12.13245


In [ ]:
train,val=train_test_split(data,test_size=0.2,random_state=1994)
test = pd.read_csv("DeepChain/test.csv")

# process sequences 
# first step : map Amino acids to integer 
# second step : convert the sequence of integer  to a list 

train.CDR1=train.CDR1.apply(lambda x: [voc_set_map[e] for e in x ])
train.CDR2=train.CDR2.apply(lambda x: [voc_set_map[e] for e in x ])
train.CDR3=train.CDR3.apply(lambda x: [voc_set_map[e] for e in x ])
train.sequence=train.sequence.apply(lambda x: [voc_set_map[e] for e in x ])
cdr1 = np.array(np.array(train[["CDR1"]].values).tolist()).reshape(len(train),-1)
cdr2 = np.array(np.array(train[["CDR2"]].values).tolist()).reshape(len(train),-1)
cdr3 = np.array(np.array(train[["CDR3"]].values).tolist()).reshape(len(train),-1)
sequence = np.array(np.array(train[["sequence"]].values).tolist()).reshape(len(train),-1)
train["CDR"]= np.concatenate((cdr1,cdr2,cdr3,sequence),axis=1).tolist()

val.CDR1=val.CDR1.apply(lambda x: [voc_set_map[e] for e in x ])
val.CDR2=val.CDR2.apply(lambda x: [voc_set_map[e] for e in x ])
val.CDR3=val.CDR3.apply(lambda x: [voc_set_map[e] for e in x ])
val.sequence=val.sequence.apply(lambda x: [voc_set_map[e] for e in x ])
cdr1 = np.array(np.array(val[["CDR1"]].values).tolist()).reshape(len(val),-1)
cdr2 = np.array(np.array(val[["CDR2"]].values).tolist()).reshape(len(val),-1)
cdr3 = np.array(np.array(val[["CDR3"]].values).tolist()).reshape(len(val),-1)
sequence = np.array(np.array(val[["sequence"]].values).tolist()).reshape(len(val),-1)
val["CDR"]= np.concatenate((cdr1,cdr2,cdr3,sequence),axis=1).tolist()


test.CDR1=test.CDR1.apply(lambda x: [voc_set_map[e] for e in x ])
test.CDR2=test.CDR2.apply(lambda x: [voc_set_map[e] for e in x ])
test.CDR3=test.CDR3.apply(lambda x: [voc_set_map[e] for e in x ])
test.sequence=test.sequence.apply(lambda x: [voc_set_map[e] for e in x ])
cdr1 = np.array(np.array(test[["CDR1"]].values).tolist()).reshape(len(test),-1)
cdr2 = np.array(np.array(test[["CDR2"]].values).tolist()).reshape(len(test),-1)
cdr3 = np.array(np.array(test[["CDR3"]].values).tolist()).reshape(len(test),-1)
sequence = np.array(np.array(test[["sequence"]].values).tolist()).reshape(len(test),-1)
test["CDR"]= np.concatenate((cdr1,cdr2,cdr3,sequence),axis=1).tolist()


/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5170: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docum

In [ ]:
class AntiBodyTransformer(nn.Module):
  def __init__(self):
    super(AntiBodyTransformer,self).__init__()
    self.cdr1EmbeddingLayer = nn.Embedding(25,48)
    self.cdr2EmbeddingLayer = nn.Embedding(25,48)
    self.cdr3EmbeddingLayer = nn.Embedding(25,48)
    self.sequenceEmbeddingLayer = nn.Embedding(25,48)
    
    # self.cdr1EmbeddingLayer.weight.requires_grad = False
    # self.cdr2EmbeddingLayer.weight.requires_grad = False
    # self.cdr3EmbeddingLayer.weight.requires_grad = False
    self.transEncoderLayer = nn.TransformerEncoderLayer(48,8)
    self.batchnorm1 = nn.BatchNorm1d(241)
    self.linear1= nn.Linear(48,1)
    self.drop1 = nn.Dropout(0.2)
    self.drop2 = nn.Dropout(0.2)
    self.linear2= nn.Linear(241,1)

  def forward(self,x):
    cdr1EmbeddingVectors = self.cdr1EmbeddingLayer(x[:,:5].long())
    cdr2EmbeddingVectors = self.cdr2EmbeddingLayer(x[:,5:12].long())
    cdr3EmbeddingVectors = self.cdr3EmbeddingLayer(x[:,12:20].long())
    sequenceEmbeddingVectors = self.sequenceEmbeddingLayer(x[:,20:].long())
    
    x= torch.cat([cdr1EmbeddingVectors,cdr2EmbeddingVectors,cdr3EmbeddingVectors,sequenceEmbeddingVectors],axis=1)
    # print(x.size())
    # return x
    atten= self.transEncoderLayer(x)
    atten= self.drop1(atten)
    out = F.leaky_relu(self.linear1(atten))
    
    out = out.view(-1,241)
    out=  self.batchnorm1(out)
    # out = self.batchnorm1(out)
    out = self.drop2(out)

    out = self.linear2(out)
    return out



In [ ]:

batch_size = 512

model = AntiBodyTransformer().to(device)
optimizer = optim.Adam(model.parameters(),lr = 0.01,weight_decay=0.0001)
npData = np.array(train[["CDR","binding"]].values)
valnp = np.array(val[["CDR","binding"]].values)
# target = npData[:,1]
def ceil(a,b):
    return -(-a//b)

n_samples = len(npData)
better_batch_size = ceil(n_samples, ceil(n_samples, batch_size))

for e in range(510):
  model.train()
  trainerrs = []
  for i in range(ceil(n_samples, better_batch_size)):
    batchx = np.array(npData[i*better_batch_size : (i+1)*better_batch_size,0].tolist())
    batchy = np.array(npData[i*better_batch_size : (i+1)*better_batch_size,1:].tolist())
    batchy = torch.Tensor(batchy.astype("float32")).to(device)
    batchx = torch.Tensor(batchx.astype("float32")).to(device)
    model.zero_grad()
    pred = model(batchx)
    # break
    err = F.mse_loss(pred,batchy)
    err.backward()
    trainerrs.append(np.sqrt(err.item()))
    optimizer.step()
  # break
  model.eval()
  valtrues = torch.Tensor([])
  valpreds = torch.Tensor([])
  for i in range(ceil(len(val), better_batch_size)):
    validTensorx = torch.Tensor(np.array(valnp[i*better_batch_size : (i+1)*better_batch_size,0].tolist()).astype("float32")).to(device)
    validTensory = torch.Tensor(np.array(valnp[i*better_batch_size : (i+1)*better_batch_size,1:].tolist()).astype("float32"))
    valpred = model(validTensorx).cpu().detach()
    valtrues = torch.cat([valtrues,validTensory])
    valpreds = torch.cat([valpreds,valpred])

  err = F.mse_loss(valpreds,valtrues)
  if np.sqrt(err.item()) < bestvalid :
    print("new best")
    torch.save(model.state_dict(),"./model.tar")
    bestvalid =np.sqrt(err.item())

  print("train ",np.mean(trainerrs), " valid",np.sqrt(err.item()))


In [ ]:
checkpoint = torch.load("./model.tar")
model.load_state_dict(checkpoint)

<All keys matched successfully>

In [ ]:
nptest = np.array(test[["CDR"]].values)
valpreds = torch.Tensor([])
for i in range(ceil(len(nptest), better_batch_size)):
  validTensorx = torch.Tensor(np.array(nptest[i*better_batch_size : (i+1)*better_batch_size,0].tolist()).astype("float32")).to(device)
  # validTensory = torch.Tensor(np.array(val[i*better_batch_size : (i+1)*better_batch_size,1:].tolist()).astype("float32"))
  valpred = model(validTensorx).cpu().detach()
  valpreds = torch.cat([valpreds,valpred])


sub=test[["ID"]].copy()
sub["binding"]=valpreds.numpy()
sub.to_csv("sub.csv",index=False)

In [ ]:
test_pred=model.predict(X_test,verbose=True)
sub=test[["ID"]].copy()
sub["binding"]=test_pred
sub.to_csv("sub.csv",index=False)